<a href="https://colab.research.google.com/github/shanilev123456/Final-Project-ds/blob/master/RMI_Data_and_GovMap_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import files
uploaded = files.upload()

Saving merged_file_beforeCleaning.csv to merged_file_beforeCleaning (1).csv


In [18]:
import pandas as pd
df = pd.read_csv('merged_file_beforeCleaning.csv')
df.head()

,Tender Number,Region,City,Purpose,Publication date,Status,Opening date,Submission deadline,Committee date,Compound Number,...,dist_to_park,is_park_within_radius,dist_to_mall,is_mall_within_radius,dist_to_supermarket,is_supermarket_within_radius,dist_to_beach,is_beach_within_radius,dist_to_place_of_worship,is_place_of_worship_within_radius
0,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,68,...,1829.452115,0.0,20090.284153,0.0,613.373476,1.0,18844.622769,0.0,1588.610909,0.0
1,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,70,...,1829.452115,0.0,20090.284153,0.0,613.373476,1.0,18844.622769,0.0,1588.610909,0.0
2,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,71,...,1829.452115,0.0,20090.284153,0.0,613.373476,1.0,18844.622769,0.0,1588.610909,0.0
3,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,15,...,15715.992511,0.0,20605.544545,0.0,11013.265756,0.0,20634.016007,0.0,2623.485180,0.0
4,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,A15,...,15715.992511,0.0,20605.544545,0.0,11013.265756,0.0,20634.016007,0.0,2623.485180,0.0


In [19]:

df = df[df['Purpose'] != "בנייה נמוכה/צמודת קרקע -לייזמים/קבלנים"]
df = df.drop(columns=['Tender Number', 'Purpose','Publication date','Status', 'Winner Name', 'Submission deadline', 'Plan','Valuation Price', 'Minimum Price', 'Development Cost', 'Compound Number', 'Opening date','X','Y'])

In [20]:

# Remove missing values

print("missing data:")
print(df.isnull().sum())

df = df[df['Final Price'].notnull()]
df = df[df['Region'].notnull()]
df = df[df['Final Price'] != "אין הצעות למתחם זה"]

numeric_columns = ['Final Price']

for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Handling missing values in numerics columns
df['Units'] = df['Units'].fillna(df['Units'].median())

df['Parcel'] = df['Parcel'].fillna("Not existing")
df['Block'] = df['Block'].fillna("Not existing")
df['Section'] = df['Section'].fillna("Not existing") # remove this

missing data:
Region                                396
City                                  396
Committee date                        396
Units                                 212
Final Price                          3207
Area m2                                 2
Parcel                                482
Block                                 221
Section                               221
dist_to_kindergarten                 2897
is_kindergarten_within_radius        2897
dist_to_school                       2897
is_school_within_radius              2897
dist_to_university                   2897
is_university_within_radius          2897
dist_to_bus_stop                     2897
is_bus_stop_within_radius            2897
dist_to_train_station                2897
is_train_station_within_radius       2897
dist_to_park                         2897
is_park_within_radius                2897
dist_to_mall                         2897
is_mall_within_radius                2897
dist_to_supermarket 

In [21]:

df['Final Price'] = df['Final Price'].replace(',', '', regex=True).astype(float)
df['Area m2'] = df['Area m2'].replace(',', '', regex=True).astype(float)
print(f"before filter Final Price: {len(df)}")

# Manual binderies
lower_bound = df['Final Price'].quantile(0.05)
upper_bound = df['Final Price'].quantile(0.95)

df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]
print(f"after filter Final Price: {len(df)}")
df = df[(df['Area m2'] >= 100) & (df['Area m2'] <= 1000)]
print(f"after filter Area m2: {len(df)}")

before filter Final Price: 6679
after filter Final Price: 6011
after filter Area m2: 5754


In [22]:
df['Committee date'] = pd.to_datetime(df['Committee date'], errors='coerce')
df['Committee_year'] = df['Committee date'].dt.year
df['Committee_month'] = df['Committee date'].dt.month

df = df.drop(columns=['Committee date'])

In [23]:

print(df['Final Price'].describe())

count    5.754000e+03
mean     5.499860e+05
std      4.872621e+05
min      3.586300e+04
25%      2.099915e+05
50%      4.130000e+05
75%      7.113462e+05
max      2.475555e+06
Name: Final Price, dtype: float64


In [24]:

df['Price per m2'] = df['Final Price'] / df['Area m2']

# Saving min- max vals of price.
price_per_m2_min = df['Price per m2'].min()
price_per_m2_max = df['Price per m2'].max()

In [25]:
import json
price_minmax = {
    "min": float(price_per_m2_min),
    "max": float(price_per_m2_max)
}

with open("price_minmax.json", "w", encoding="utf-8") as f:
    json.dump(price_minmax, f, ensure_ascii=False, indent=2)

In [26]:
numeric_columns = ['Units','Final Price','Area m2','Price per m2','dist_to_kindergarten','dist_to_school','dist_to_university',
    'dist_to_bus_stop','dist_to_train_station','dist_to_park',
    'dist_to_mall','dist_to_supermarket','dist_to_beach',
    'dist_to_place_of_worship']

scaler_dict = {}

for col in numeric_columns:
    if col not in df.columns:
        print(f"Warning: Column '{col}' not found, skipping.")
        continue

    col_min = df[col].min()
    col_max = df[col].max()
    if col_min == col_max:
        print(f"Warning: column '{col}' has the same min ({col_min}) and max. Setting all scaled to 0.")
        df[col] = 0.0
    else:
        df[col] = (df[col] - col_min) / (col_max - col_min)

    scaler_dict[col] = {
        "min": float(col_min),
        "max": float(col_max)
    }

block_one_hot = pd.get_dummies(df['Block'], prefix='Block')
# section_one_hot = pd.get_dummies(df['Section'], prefix='Section')
# parcel_one_hot = pd.get_dummies(df['Parcel'], prefix='Parcel')
committee_month_one_hot = pd.get_dummies(df['Committee_month'], prefix='Committee_month')
committee_year_one_hot = pd.get_dummies(df['Committee_year'], prefix='Committee_year')


df = df.drop(['Block', 'Section', 'Parcel', 'Committee_month', 'Committee_year'], axis=1)

# df = pd.concat([df, block_one_hot, section_one_hot, parcel_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)
df = pd.concat([df, block_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)

with open("scaler_minmax.json", "w", encoding="utf-8") as f:
    json.dump(scaler_dict, f, ensure_ascii=False, indent=2)

normalized_columns = ['Final Price', 'Area m2', 'Units','dist_to_kindergarten', 'dist_to_school', 'dist_to_university',
    'dist_to_bus_stop', 'dist_to_train_station', 'dist_to_park',
    'dist_to_mall', 'dist_to_supermarket', 'dist_to_beach',
    'dist_to_place_of_worship']

In [27]:
df = pd.get_dummies(df, columns=['Region', 'City'], prefix=['Region', 'City']) # קידוד 0-1 לערים ומחוזות
df = df.replace({True: 1, False: 0})

<ipython-input-27-be404e8d7d5b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [28]:
city_columns = [col for col in df.columns if col.startswith('City_')]

cities = set()
for col in city_columns:
    city_name = col.replace('City_', '').split(',')[0].strip()
    cities.add(city_name)

In [29]:
southern_base_cities = [
    "אופקים", "אשקלון", "באר גנים", "באר שבע", "גדרה", "גן יבנה",
    "חלץ", "יבנה", "מיתר", "להבים", "דימונה", "ניצן", "נתיבות",
    "עומר", "קרית גת", "קרית מלאכי", "שדרות"
]

southern_city_columns = []
for city in southern_base_cities:
    matches = [col for col in df.columns if col.startswith(f"City_{city}")]
    southern_city_columns.extend(matches)

df = df[df[southern_city_columns].any(axis=1)]

all_city_columns = [col for col in df.columns if col.startswith("City_")]
columns_with_data = [col for col in all_city_columns if df[col].sum() > 0]

columns_to_drop = set(all_city_columns) - set(columns_with_data)
df = df.drop(columns=columns_to_drop)

print(columns_with_data)

['City_אופקים , הפארק', 'City_אופקים , חורשת נח', 'City_אופקים , רמת שקד', 'City_אשקלון , הרצוג', 'City_אשקלון , הרצוג נכים', 'City_באר גנים', 'City_באר גנים , נכים', 'City_באר שבע , הרקפות', 'City_באר שבע , כלניות', 'City_באר שבע , כלניות לנכים', 'City_באר שבע , נאות לון', 'City_באר שבע , נווה נוי', 'City_באר שבע , רמות', 'City_באר שבע , רקפות', 'City_גן יבנה', 'City_גן יבנה , גן יבנה', 'City_חלץ', 'City_חלץ , מושב חלץ', 'City_יבנה , נכים', 'City_יבנה , רמות ויצמן', 'City_להבים', 'City_להבים , שרונית', 'City_מיתר , יישוב כרמית', 'City_מיתר , כלניות', 'City_מיתר , כלניות נכים', 'City_מיתר , כרמית', 'City_ניצן', 'City_נתיבות', 'City_נתיבות , מעלות הנחל', 'City_נתיבות , נוה שרון', 'City_נתיבות , נווה שרון', 'City_נתיבות , רמות יורם', 'City_נתיבות , רמות מאיר', 'City_נתיבות , שערי העיר', 'City_עומר', 'City_עומר , עומר', 'City_עומר , עמר נכים', 'City_קרית גת , כרמי גת', 'City_קרית גת , כרמי גת נכים', 'City_קרית גת , מערבית', 'City_קרית גת , מתחם 01', 'City_קרית גת , רובע הבנים', "City_קרית

In [30]:

df=df.drop(['Final Price'],axis=1)

In [31]:
df.to_csv("cleaned_data.csv", index=False)

files.download("/content/cleaned_data.csv")
files.download("/content/scaler_minmax.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>